In [1]:
import pandas as pd
import random
import os
import numpy as np
import matplotlib.pyplot as plt
import warnings; warnings.filterwarnings(action='ignore')
import pickle
import time
from sklearn.ensemble import GradientBoostingClassifier
from scipy.stats import randint
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from bayes_opt import BayesianOptimization
from xgboost import XGBClassifier # model 
from xgboost import plot_importance
from sklearn.ensemble import VotingClassifier
from lightgbm import LGBMClassifier
from scipy.stats import randint
import sklearn.svm as svm


from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

In [246]:
import sys   
!{sys.executable} -m pip install time

ERROR: Could not find a version that satisfies the requirement time (from versions: none)
ERROR: No matching distribution found for time


In [119]:
def seed_everything(seed):
    random.seed(seed) #파이썬 자체 모듈 random 모듈의 시드 고정
    os.environ['PYTHONHASHSEED'] = str(seed) 
    np.random.seed(seed) #넘파이를 사용할 경우
#     torch.manual_seed(seed)
#     torch.cuda.manual_seed(seed)
#     torch.backends.cudnn.deterministic = True
    
seed_everything(37) # Seed 고정

# Data Load

In [2]:
#데이터프레임 불러오기
train_df = pd.read_csv('./train_fillnan_t23.csv')
test_df = pd.read_csv('./test_fillnan_t23.csv')

In [3]:
sub1 = pd.read_csv('./sample_submission.csv')
sub2 = pd.read_csv('./sample_submission.csv')

In [11]:
sub3 = pd.read_csv('./sample_submission.csv')
sub4 = pd.read_csv('./sample_submission.csv')

# Data Preprocessing

PRODUCT_CODE 별로 LINE 별로 데이터 나누기

1.train_df

In [122]:
train_df135 = train_df[train_df['LINE']=='T010305'] 
train_df136 = train_df[train_df['LINE']=='T010306'] 
train_df534 = train_df[train_df['LINE']=='T050304']
train_df537 = train_df[train_df['LINE']=='T050307']
train_df1034 = train_df[train_df['LINE']=='T100304']
train_df1036 = train_df[train_df['LINE']=='T100306']

2.test_df

In [123]:
test_df135 = test_df[test_df['LINE']=='T010305'] 
test_df136 = test_df[test_df['LINE']=='T010306'] 
test_df534 = test_df[test_df['LINE']=='T050304'] 
test_df537 = test_df[test_df['LINE']=='T050307'] 
test_df1034 = test_df[test_df['LINE']=='T100304'] 
test_df1036 = test_df[test_df['LINE']=='T100306'] 

In [124]:
# 분명 제품 별로 측정하는 값 종류가 다를 것이다. 그래도 아예 없는 값으로 만들어주기 위해 -100, -200을 넣는 것 보다 그냥 다른 제품의 것을 넣어보자

In [144]:
#값이 있어야 하는 결측치 채우기함수
def fill_nan_med(df): #미디언으로 해주는 이유는 yclass가 0인건 55 1인건 59 이런 식으로 규칙이 존재하기 때문에 함부로 평균을 내면 안될 것 같음
    val_lst = []
    for col in df.columns[-2875:]:
        val = df[col].median()
        if val == 'NaN':
            val = -100
        val_lst.append(val)
        if df[col].isnull().sum() == len(df): #모든 value가 결측치인 행
            df[col] = -100
        
        elif df[col].isnull().sum() < len(df): #한 행에서 결측치가 하나라도 있다면
            nan_lst = list(df[df[col].isnull()==True].index)
            for j in nan_lst: #j들은 value값이 nan인 index들.
                df[col][j] = val
    fill_df = df
    return fill_df, val_lst

In [145]:
#값이 있어야 하는 결측치 채우기함수
def fill_nan_med_ts(df, val_lst):
    for i, col in enumerate(df.columns[-2875:]):
        if df[col].isnull().sum() > 0:
            val = val_lst[i]
            nan_lst = list(df[df[col].isnull()==True].index)
            for j in nan_lst:
                df[col][j] = val
    fill_df = df
    return fill_df

In [146]:
start = time.time()
train_df135, val_135 = fill_nan_med(train_df135)
train_df136, val_136 = fill_nan_med(train_df136)
train_df534, val_534 = fill_nan_med(train_df534)
train_df537, val_537 = fill_nan_med(train_df537)
train_df1034, val_1034 = fill_nan_med(train_df1034)
train_df1036, val_1036 = fill_nan_med(train_df1036)
end = time.time()
print(f'time elapsed: {end-start}')

time elapsed: 10.58210301399231


In [147]:
val_534[2628]

-100.0

In [148]:
train_df = pd.concat([train_df135, train_df136, train_df534, train_df537, train_df1034, train_df1036], axis = 0)
train_df = train_df.sort_index(ascending=True)


In [149]:
start = time.time()
test_df135 = fill_nan_med_ts(test_df135, val_135)
test_df136 = fill_nan_med_ts(test_df136, val_136)
test_df534 = fill_nan_med_ts(test_df534, val_534)
test_df537 = fill_nan_med_ts(test_df537, val_537)
test_df1034 = fill_nan_med_ts(test_df1034, val_1034)
test_df1036 = fill_nan_med_ts(test_df1036, val_1036)
end = time.time()
print(f'time elapsed: {end-start}')

time elapsed: 1.5811901092529297


In [150]:
test_df = pd.concat([test_df135, test_df136, test_df534, test_df537, test_df1034, test_df1036], axis = 0)
test_df = test_df.sort_index(ascending=True)
test_df

,PRODUCT_ID,TIMESTAMP,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
0,TEST_000,2022-09-09 2:01,T100306,T_31,2.0,94.0,0.0,45.0,10.0,0.0,...,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0
1,TEST_001,2022-09-09 2:09,T100304,T_31,2.0,93.0,0.0,45.0,11.0,0.0,...,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0
2,TEST_002,2022-09-09 8:42,T100304,T_31,2.0,95.0,0.0,45.0,11.0,0.0,...,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0
3,TEST_003,2022-09-09 10:56,T010305,A_31,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,...,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0
4,TEST_004,2022-09-09 11:04,T010306,A_31,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,...,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305,TEST_305,2022-11-05 11:18,T100306,T_31,2.0,91.0,0.0,45.0,10.0,0.0,...,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0
306,TEST_306,2022-11-05 16:39,T100304,T_31,2.0,96.0,0.0,45.0,11.0,0.0,...,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0
307,TEST_307,2022-11-05 16:47,T100306,T_31,2.0,91.0,0.0,45.0,10.0,0.0,...,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0
308,TEST_308,2022-11-05 20:53,T100306,T_31,2.0,95.0,0.0,45.0,10.0,0.0,...,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0


In [152]:
test_df534.iloc[:,2628:]

,X_2625,X_2626,X_2627,X_2628,X_2629,X_2630,X_2631,X_2632,X_2633,X_2634,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
7,-100,-100,-100,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,...,57.74,52.51,54.45,57.99,63.16,1.0,-100,-100,-100,-100
8,-100,-100,-100,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,...,53.18,56.02,41.79,47.94,60.72,1.0,-100,-100,-100,-100
14,-100,-100,-100,49.7,50.9,50.9,51.8,59.4,52.6,52.6,...,49.77,54.93,49.85,48.62,63.18,1.0,-100,-100,-100,-100
35,-100,-100,-100,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,...,55.22,58.14,48.93,56.16,64.25,1.0,-100,-100,-100,-100
41,-100,-100,-100,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,...,49.67,45.00,49.81,55.19,67.17,1.0,-100,-100,-100,-100
42,-100,-100,-100,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,...,51.30,47.66,60.00,53.75,62.58,1.0,-100,-100,-100,-100
52,-100,-100,-100,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,...,54.06,51.18,52.17,57.75,63.94,1.0,-100,-100,-100,-100
53,-100,-100,-100,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,...,54.04,51.25,54.98,57.22,63.33,1.0,-100,-100,-100,-100
54,-100,-100,-100,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,...,55.72,57.57,42.08,55.19,66.06,1.0,-100,-100,-100,-100
61,-100,-100,-100,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,...,53.81,56.14,51.33,56.21,65.14,1.0,-100,-100,-100,-100


### Feature Selection

outlier 이상체 제거한 최종 데이터 프레임 저장하기 & 모든 value가 결측치이거나 0.0인 행 지운 test data도 저장하기

In [153]:
train_df.to_csv('./train_fillnan_t23.csv', index = False) 

In [154]:
test_df.to_csv('./test_fillnan_t23.csv', index = False) #traindf의 수치데이터를 testdf에 다시 적용해 전처리

모델링을 하기 앞서, 독립변수 X와 종속변수 y를 설정해야한다.

**학습에 사용할 변수 X와 예측할 변수 y를 분리**

In [111]:
# train_y = train_df['Y_Class']

In [4]:
train_y = train_df['Y_Class']

학습에 쓰이지 않을 column들을 제거한다.

In [5]:
train_x = train_df.drop(columns = ['PRODUCT_ID', 'TIMESTAMP', 'Y_Class', 'Y_Quality'])
#모델 학습이 끝나고 예측에 쓰일 test데이터
test_x = test_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP'])

In [186]:
# train_x = train_df1036.drop(columns = ['PRODUCT_ID', 'TIMESTAMP', 'Y_Class', 'Y_Quality', 'LINE'])
# #모델 학습이 끝나고 예측에 쓰일 test데이터
# test_x = test_df1036.drop(columns=['PRODUCT_ID', 'TIMESTAMP', 'LINE'])

In [187]:
p1 = test_x.index.tolist()
p1

[0,
 16,
 18,
 19,
 21,
 26,
 29,
 31,
 33,
 34,
 43,
 44,
 45,
 46,
 48,
 50,
 56,
 57,
 67,
 74,
 76,
 78,
 81,
 85,
 91,
 94,
 96,
 98,
 100,
 102,
 104,
 105,
 107,
 110,
 111,
 112,
 118,
 120,
 121,
 124,
 125,
 127,
 133,
 136,
 139,
 140,
 143,
 146,
 148,
 149,
 150,
 152,
 153,
 154,
 156,
 159,
 160,
 162,
 164,
 165,
 166,
 167,
 168,
 171,
 173,
 175,
 178,
 181,
 182,
 184,
 185,
 186,
 188,
 190,
 192,
 194,
 197,
 199,
 201,
 203,
 206,
 208,
 210,
 211,
 213,
 215,
 216,
 218,
 220,
 223,
 224,
 225,
 227,
 229,
 230,
 231,
 232,
 235,
 236,
 237,
 239,
 240,
 242,
 247,
 257,
 259,
 261,
 262,
 265,
 266,
 267,
 268,
 269,
 270,
 271,
 273,
 274,
 275,
 276,
 278,
 279,
 288,
 289,
 290,
 291,
 295,
 296,
 301,
 305,
 307,
 308,
 309]

범주형 데이터를 수치 데이터로 전환하기 위해 LabelEncoder 활용

In [6]:
# qualitative to quantitative
qual_col = ['LINE', 'PRODUCT_CODE']

for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train_x[i]) #원래 column 값을 기준으로 fit.
    train_x[i] = le.transform(train_x[i]) #수치화, 수치로 변형

    for label in np.unique(test_x[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test_x[i] = le.transform(test_x[i]) 
print('Done.')

Done.


In [112]:
test_x.isnull().sum()[test_x.isnull().sum() > 0]

X_2628    12
X_2629    12
X_2630    12
X_2631    12
X_2632    12
          ..
X_2696    12
X_2697    12
X_2698    12
X_2699    12
X_2844    12
Length: 73, dtype: int64

### Modeling

1.모델 선택 - sklearn라이브러리 활용 - RandomForest 

2.모델 학습 - train_df를 활용하여 1번에서 정의한 모델로 학습

3.예측 - 학습된 모델을 바탕으로 test 데이터를 예측

4.정답 파일 생성 - 정답 파일 생성 및 제출 필요(경진대회를 위해 필요한 과정.)

subRF_r = pd.read_csv('./sample_submission.csv')
subXGB_r = pd.read_csv('./sample_submission.csv')
sub_v = pd.read_csv('./sample_submission.csv')

# Randomsearch

1.GBM randomsearch

2.Randomforestregressor & SVM

### GBM randomsearch

In [7]:
param_distribs = {
    'n_estimators': randint(low=1, high=200),
    'min_samples_split': randint(low = 1, high = 3),
    'min_samples_leaf': randint(low = 1, high = 10)
}

GBM= GradientBoostingClassifier(random_state = 37)
start = time.time()
rand_cv = RandomizedSearchCV(GBM, 
                            param_distributions = param_distribs,
                            cv = 5, 
                            n_iter = 50,
                            scoring = 'f1_macro',
                            n_jobs = -1,
                            verbose = 3)
rand_cv.fit(train_x, train_y)
preds = rand_cv.predict(test_x)
end = time.time()
sub1['Y_Class'] = preds
sub1.to_csv('./t24_GBM_random_redata.csv', index=False) #median으로 채우고 나머지 -100으로 채우고 train의 수치를 test에 적용

print(f'최적 하이퍼 파라미터: {rand_cv.best_params_}')
print(f'최고 예측 정확도: {(rand_cv.best_score_)*100:.4f}')
print(f'time elapsed: {end-start}')

Fitting 5 folds for each of 50 candidates, totalling 250 fits
최적 하이퍼 파라미터: {'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 152}
최고 예측 정확도: 43.6473
time elapsed: 647.9960780143738
[CV 2/5] END min_samples_leaf=3, min_samples_split=2, n_estimators=192;, score=0.549 total time= 1.1min
[CV 4/5] END min_samples_leaf=1, min_samples_split=2, n_estimators=137;, score=0.580 total time=  45.8s
[CV 1/5] END min_samples_leaf=3, min_samples_split=2, n_estimators=130;, score=0.483 total time=  40.9s
[CV 5/5] END min_samples_leaf=3, min_samples_split=2, n_estimators=130;, score=0.136 total time=  43.9s
[CV 5/5] END min_samples_leaf=4, min_samples_split=2, n_estimators=19;, score=0.187 total time=   6.4s
[CV 3/5] END min_samples_leaf=7, min_samples_split=2, n_estimators=90;, score=0.410 total time=  28.2s
[CV 1/5] END min_samples_leaf=6, min_samples_split=2, n_estimators=187;, score=0.470 total time=  56.8s
[CV 4/5] END min_samples_leaf=5, min_samples_split=2, n_estimators=196;, score=

In [192]:
sub1['Y_Class'] = preds['y_class']
sub1.to_csv('./t23_sep_data_com_redata.csv', index = False)

In [12]:
sub1.describe()

,Y_Class
count,310.000000
mean,1.025806
std,0.468398
min,0.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,2.000000


### XGB randomsearch

In [10]:
param_distribs = {
    'min_child_weight': randint(low = 1, high = 10),
    'gamma': randint(low = 1, high = 10),
    'max_depth': randint(low = 5, high = 10),
    'lambda' : randint(low = 1, high = 10),
    'alpha' : randint(low = 1, high = 10)
}
start = time.time()
xgboost_r = XGBClassifier(seed=37).fit(train_x,train_y)
rand_cv = RandomizedSearchCV(xgboost_r, 
                            param_distributions=param_distribs,
                            cv = 5, 
                            n_iter = 50,
                            scoring = 'f1_macro',
                            n_jobs = -1,
                            verbose=3)
rand_cv.fit(train_x, train_y)
preds = rand_cv.predict(test_x)
end = time.time()
sub2['Y_Class'] = preds
sub2.to_csv('./t24_XGB_random_redata.csv', index=False)

print(f'최적 하이퍼 파라미터: {rand_cv.best_params_}')
print(f'최고 예측 정확도: {(rand_cv.best_score_)*100:.4f}')
print(f'time elapsed: {end-start}')

Fitting 5 folds for each of 50 candidates, totalling 250 fits
최적 하이퍼 파라미터: {'alpha': 9, 'gamma': 1, 'lambda': 9, 'max_depth': 7, 'min_child_weight': 4}
최고 예측 정확도: 41.3334
time elapsed: 2092.5014369487762


In [14]:
sub2.describe()

,Y_Class
count,310.000000
mean,0.941935
std,0.364060
min,0.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,2.000000


### RandomForest randomsearch

In [13]:
param_distribs = {
    'n_estimators' : randint(low=1, high=200),
    'min_samples_split': randint(low = 1, high = 10),
    'min_samples_leaf': randint(low = 1, high = 17),
    'max_depth': randint(low = 1, high = 8)
}
start = time.time()
rf_r = RandomForestClassifier(random_state=37)
rand_cv = RandomizedSearchCV(rf_r, 
                            param_distributions=param_distribs,
                            cv = 5, 
                            n_jobs = -1,
                            verbose=3)
rand_cv.fit(train_x, train_y)
preds = rand_cv.predict(test_x)
end = time.time()
print('time elapsed:', end - start)
print(f'최적 하이퍼 파라미터: {rand_cv.best_params_}')
print(f'최고 예측 정확도: {(rand_cv.best_score_)*100:.4f}')
sub3['Y_Class'] = preds
sub3.to_csv('./t24_RandomForest_random_redata.csv', index=False)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
time elapsed: 4.168781995773315
최적 하이퍼 파라미터: {'max_depth': 1, 'min_samples_leaf': 15, 'min_samples_split': 4, 'n_estimators': 136}
최고 예측 정확도: 67.0602
[CV 4/5] END alpha=3, gamma=3, lambda=2, max_depth=5, min_child_weight=1;, score=0.386 total time=  55.9s
[CV 3/5] END alpha=8, gamma=3, lambda=1, max_depth=5, min_child_weight=7;, score=0.489 total time=  50.6s
[CV 3/5] END alpha=8, gamma=6, lambda=9, max_depth=9, min_child_weight=8;, score=0.452 total time= 1.2min
[CV 5/5] END alpha=3, gamma=8, lambda=1, max_depth=9, min_child_weight=3;, score=0.271 total time= 1.5min
[CV 4/5] END alpha=3, gamma=2, lambda=3, max_depth=8, min_child_weight=4;, score=0.433 total time= 1.2min
[CV 3/5] END alpha=2, gamma=7, lambda=3, max_depth=5, min_child_weight=8;, score=0.452 total time=  55.1s
[CV 4/5] END alpha=3, gamma=9, lambda=4, max_depth=6, min_child_weight=3;, score=0.303 total time= 1.1min
[CV 4/5] END alpha=5, gamma=3, lambda=9, max_de

[CV 2/5] END alpha=3, gamma=1, lambda=2, max_depth=9, min_child_weight=5;, score=0.399 total time= 1.1min
[CV 1/5] END alpha=8, gamma=6, lambda=9, max_depth=9, min_child_weight=8;, score=0.318 total time= 1.2min
[CV 4/5] END alpha=4, gamma=4, lambda=9, max_depth=5, min_child_weight=5;, score=0.299 total time=  50.6s
[CV 2/5] END alpha=8, gamma=5, lambda=3, max_depth=7, min_child_weight=8;, score=0.385 total time= 1.1min
[CV 1/5] END alpha=3, gamma=2, lambda=3, max_depth=8, min_child_weight=4;, score=0.386 total time= 1.2min
[CV 4/5] END alpha=3, gamma=7, lambda=2, max_depth=7, min_child_weight=7;, score=0.299 total time= 1.2min
[CV 1/5] END alpha=4, gamma=1, lambda=7, max_depth=5, min_child_weight=6;, score=0.374 total time=  50.1s
[CV 1/5] END alpha=5, gamma=3, lambda=9, max_depth=6, min_child_weight=5;, score=0.373 total time= 1.0min
[CV 4/5] END alpha=5, gamma=7, lambda=3, max_depth=8, min_child_weight=8;, score=0.300 total time= 1.2min
[CV 2/5] END alpha=5, gamma=3, lambda=8, max_d

[CV 1/5] END alpha=3, gamma=3, lambda=2, max_depth=5, min_child_weight=1;, score=0.392 total time=  54.5s
[CV 4/5] END alpha=3, gamma=1, lambda=2, max_depth=9, min_child_weight=5;, score=0.439 total time= 1.1min
[CV 3/5] END alpha=4, gamma=4, lambda=9, max_depth=5, min_child_weight=5;, score=0.343 total time=  49.5s
[CV 2/5] END alpha=3, gamma=8, lambda=1, max_depth=9, min_child_weight=3;, score=0.349 total time= 1.5min
[CV 3/5] END alpha=3, gamma=2, lambda=3, max_depth=8, min_child_weight=4;, score=0.368 total time= 1.1min
[CV 5/5] END alpha=3, gamma=7, lambda=2, max_depth=7, min_child_weight=7;, score=0.271 total time= 1.1min
[CV 5/5] END alpha=3, gamma=9, lambda=4, max_depth=6, min_child_weight=3;, score=0.270 total time= 1.1min
[CV 5/5] END alpha=5, gamma=3, lambda=9, max_depth=6, min_child_weight=5;, score=0.357 total time= 1.0min
[CV 2/5] END alpha=2, gamma=3, lambda=1, max_depth=6, min_child_weight=1;, score=0.512 total time= 1.1min
[CV 5/5] END alpha=5, gamma=3, lambda=8, max_d

In [19]:
rf = RandomForestClassifier(random_state=37, max_depth = 5, min_samples_leaf = 1, min_samples_split = 7, n_estimators = 55)
rf.fit(train_x, train_y)
preds = rf.predict(test_x)
sub4['Y_Class'] = preds
sub4.to_csv('./t24_RandomForest_putparam_redata.csv', index = False)

In [20]:
sub4.describe()

,Y_Class
count,310.000000
mean,1.041935
std,0.324086
min,0.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,2.000000


In [15]:
sub3.describe() #Randomforest는 어떤 데이터를 넣든 모두 1로 나온다 다음부터 x

,Y_Class
count,310.0
mean,1.0
std,0.0
min,1.0
25%,1.0
50%,1.0
75%,1.0
max,1.0


In [18]:
# #train_df_0 index 재설정
# train_df_0 = train_df_0.reset_index(inplace = False, drop = True)
# train_df_0

,PRODUCT_ID,Y_Class,Y_Quality,TIMESTAMP,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
0,TRAIN_022,0,0.517719,2022-06-14 8:53,T100304,T_31,2.0,102.0,0.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TRAIN_023,0,0.519090,2022-06-14 9:01,T100304,T_31,2.0,102.0,0.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TRAIN_028,0,0.521249,2022-06-19 20:26,T010305,A_31,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TRAIN_034,0,0.521524,2022-06-21 17:36,T050304,A_31,NaN,NaN,NaN,NaN,...,55.03,52.24,55.33,57.49,67.31,1.0,NaN,NaN,NaN,NaN
4,TRAIN_066,0,0.524408,2022-06-25 21:38,T010305,A_31,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,TRAIN_583,0,0.522340,2022-09-05 8:34,T050304,A_31,NaN,NaN,NaN,NaN,...,51.71,59.64,54.61,57.05,63.18,1.0,NaN,NaN,NaN,NaN
84,TRAIN_584,0,0.519519,2022-09-05 11:09,T010305,A_31,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
85,TRAIN_585,0,0.515214,2022-09-05 11:17,T010306,A_31,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86,TRAIN_594,0,0.524022,2022-09-08 22:38,T050304,A_31,NaN,NaN,NaN,NaN,...,49.47,53.07,50.89,55.10,66.49,1.0,NaN,NaN,NaN,NaN
